In [3]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

def ipex_jit_test():
    model = models.resnet18()
    inputs = torch.randn(5, 3, 224, 224)
    model.eval()
    model = torch.jit.trace(model, torch.randn(5, 3, 224, 224))
    model = torch.jit.freeze(model)

    def foo(inputs, p=None):
        for i in range(30):
            model(inputs)

    with torch.profiler.profile(
                activities=[torch.profiler.ProfilerActivity.CPU],
                record_shapes=True,
            ) as p:
        foo(inputs, p=p)
    print(p.key_averages().table(sort_by="cpu_time_total", row_limit=10))

ipex_jit_test()
print("=======================================\n")
import intel_extension_for_pytorch as ipex
ipex_jit_test()

STAGE:2023-08-13 21:31:14 1649545:1649545 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-08-13 21:31:15 1649545:1649545 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-08-13 21:31:15 1649545:1649545 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


---------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            forward         1.53%      12.419ms        99.65%     806.772ms      26.892ms            30  
             ipex_prepack::convolution_add_relu_run        45.41%     367.687ms        45.41%     367.687ms       1.585ms           232  
                 ipex_prepack::convolution_relu_run        35.60%     288.221ms        35.76%     289.548ms       1.109ms           261  
                                     dil_max_pool2d         6.08%      49.216ms         6.20%      50.196ms       1.731ms            29  
                      ipex_prepack

STAGE:2023-08-13 21:31:16 1649545:1649545 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-08-13 21:31:17 1649545:1649545 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-08-13 21:31:17 1649545:1649545 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


---------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            forward         1.52%      11.727ms        99.56%     766.032ms      25.534ms            30  
             ipex_prepack::convolution_add_relu_run        47.23%     363.385ms        47.23%     363.385ms       1.566ms           232  
                 ipex_prepack::convolution_relu_run        34.92%     268.682ms        35.08%     269.888ms       1.034ms           261  
                                     dil_max_pool2d         5.34%      41.065ms         5.46%      42.003ms       1.448ms            29  
                      ipex_prepack